## Лабораторная работа 4: topic modeling

В данной лабораторной работе мы попытаемся обучить LDA-модель topic-моделингу на двух принципиально различных корпусах. 

В первой части вы познакомитесь с новыми возможностями библиотеки gensim, а также с возможностями парсинга в языке Python. Во второй части вам предстоит самостоятельно обучить LDA-модель и оценить качество её работы.

### Часть 1: topic modeling уровня /b/

Краеугольным камнем в машинном обучений в целом, и в NLP в частности, является выбор датасетов. Доселе мы использовали только стандартные, многократно обкатанные датасеты, но сегодня попробуем собрать свой. Практика работы с сырыми, необработанными данными весьма полезно! Заодно изучим возможности парсеров в Питоне.

Давайте напишем парсер, собирающий информацию о сообщения с русскомязычного анононимного форума (имиджборды) "Двач" ("Сосач", "Хиккач", если вам угодно). Двач, как и всякая имиджборда разделён на разделы (доски, борды), посвященные различным тематикам -- аниме, видеоигры, литература, религия... Каждая доска состоит из тем (тредов, топиков), которые создаются анонимными (при их желании) пользователями. Каждый тред посвящен обсуждению какого-то конкретного вопроса.

У некоторых разделов есть раздел архив, располагается он по адресу https://2ch.hk/(название раздела)/arch/, например для раздела музыка -- https://2ch.hk/mu/arch/. Если у вас есть минимальные навыки в языке html, а также если вы изучили документацию встроенного класса HTMLParser, то вам будет несложно написать два парсера.

Первый парсер (ArchiveParser) парсит HTML-страницу архива доски, вытягивает из неё ссылки на заархивированные треды, и скармливает их второму парсеру.

Второй парсер (ThreadParser) парсит HTML-страницу заархивированного треда, вытягивает из неё сообщения, складывает их вместе и собирает.

In [1]:
import time
import urllib.request
from html.parser import HTMLParser
from gensim.utils import simple_preprocess

def get_value_by_key(attrs, key):
    for (k, v) in attrs:
        if(k == key):
            return v;
    return None

class ArchiveParser(HTMLParser):
    flag = False
    threads = []
    limit = 200
    def handle_starttag(self, tag, attrs):
        if(self.limit > 0):
            if(tag == 'div'):
                cl = get_value_by_key(attrs, 'class')
                if (cl == 'box-data'):
                    self.flag = True;
            if(self.flag == True and tag == 'a'):
                href = get_value_by_key(attrs, 'href')
                if(len(href)>20):
                    print(href)
                    print(self.limit)
                    thread = parse_thread('https://2ch.hk' + href)
                    if(len(thread) > 10):
                        self.threads.append(thread)
                        self.limit = self.limit - 1
                    thread = []
        

    def handle_endtag(self, tag):
        if(tag == 'div'):
            self.flag = False;

    def handle_data(self, data):
        1+1
        
    def get_threads(self):
        return self.threads
    
    def clean(self):
        self.threads = []
        
parser = ArchiveParser()

def parse_archive(board = '/b/', page_number = 0):
    lines = []
    link = 'https://2ch.hk' + board + 'arch/' + str(page_number) +'.html'
    print(link)
    parser.limit = 100
    url = urllib.request.urlopen(link)
    for line in url.readlines():
        lines.append(line.decode('utf-8'))
    for line in lines:
        parser.feed(line)
    res = parser.get_threads()
    parser.clean()
    return res

In [3]:
class ThreadParser(HTMLParser):
    flag = False
    message = []
    messages = []
            
    def handle_starttag(self, tag, attrs):
        if(tag == 'blockquote'):
            self.flag = True;
            self.message = []
        

    def handle_endtag(self, tag):
        if(tag == 'blockquote'):
            self.flag = False
            if(self.message != []):
                self.messages.append(self.message)
            self.message = []

    def handle_data(self, data):
        if(self.flag):
            self.message.extend(simple_preprocess(data))
            
    def get_messages(self):
        return self.messages
    
    def clear_messages(self):
        flag = False
        self.message = []
        self.messages = []

t_parser = ThreadParser()

def parse_thread (link):
    url = urllib.request.urlopen(link)
    lines = []
    for line in url.readlines():
        lines.append(line.decode('utf-8', errors='ignore'))
    for line in lines:
        t_parser.feed(line)
    res = t_parser.get_messages()
    t_parser.clear_messages()
    #print(res)
    return res

Весьма много кода, верно? Если не потерялись, могли заметить функцию parse_archive, которая парсит страницу архива по доске и номеру страницы.


$\textbf{Задание.}$
Давайте применим её к каким-нибудь доскам. Выберите две доски двача, имеющие архив и скачайте архивы функцией parse_archive.

In [4]:
boards = ['/me/', '/mu/'] #TODO: напишите название досок в формате /'доска'/, например /mu/ для Музыки
threads_by_topic = [parse_archive(board=board) for board in boards]

https://2ch.hk/me/arch/0.html
/me/arch/2016-03-28/res/443998.html
100
/me/arch/2016-03-28/res/443998.html
100
/me/arch/2016-03-28/res/443953.html
100
/me/arch/2016-03-28/res/443857.html
100
/me/arch/2016-03-28/res/443697.html
100
/me/arch/2016-03-28/res/443694.html
100
/me/arch/2016-03-28/res/443689.html
100
/me/arch/2016-03-28/res/443688.html
100
/me/arch/2016-05-01/res/443644.html
100
/me/arch/2016-03-27/res/443640.html
99
/me/arch/2016-03-27/res/443613.html
99
/me/arch/2016-03-27/res/443524.html
99
/me/arch/2016-03-27/res/443514.html
99
/me/arch/2016-03-27/res/443490.html
98
/me/arch/2016-03-27/res/443445.html
98
/me/arch/2016-03-27/res/443444.html
98
/me/arch/2016-03-27/res/443401.html
98
/me/arch/2016-03-27/res/443354.html
98
/me/arch/2016-03-27/res/443330.html
98
/me/arch/2016-03-27/res/443277.html
98
/me/arch/2016-03-26/res/443240.html
98
/me/arch/2016-03-26/res/443067.html
98
/me/arch/2016-03-26/res/443019.html
98
/me/arch/2016-05-17/res/442975.html
98
/me/arch/2016-03-26/res/4

/me/arch/2016-03-04/res/433233.html
30
/me/arch/2016-03-04/res/433138.html
30
/me/arch/2016-03-04/res/433123.html
30
/me/arch/2016-03-04/res/433080.html
29
/me/arch/2016-03-04/res/433044.html
29
/me/arch/2016-03-04/res/433026.html
29
/me/arch/2016-03-04/res/433021.html
29
/me/arch/2016-03-04/res/432992.html
28
/me/arch/2017-01-06/res/432983.html
28
/me/arch/2016-03-04/res/432980.html
27
/me/arch/2016-03-04/res/432967.html
27
/me/arch/2016-03-04/res/432966.html
27
/me/arch/2016-03-03/res/432820.html
27
/me/arch/2016-03-03/res/432792.html
26
/me/arch/2016-03-03/res/432705.html
26
/me/arch/2016-03-03/res/432670.html
26
/me/arch/2016-03-03/res/432503.html
26
/me/arch/2016-03-02/res/432463.html
25
/me/arch/2016-03-02/res/432452.html
24
/me/arch/2016-03-02/res/432325.html
24
/me/arch/2016-03-02/res/432299.html
24
/me/arch/2016-03-02/res/432283.html
23
/me/arch/2016-03-02/res/432256.html
23
/me/arch/2016-03-02/res/432164.html
23
/me/arch/2016-03-02/res/432160.html
22
/me/arch/2016-03-02/res/4

/mu/arch/2016-03-13/res/1268957.html
44
/mu/arch/2016-03-13/res/1268952.html
43
/mu/arch/2016-03-13/res/1268945.html
42
/mu/arch/2016-03-13/res/1268918.html
42
/mu/arch/2016-03-13/res/1268910.html
42
/mu/arch/2016-03-13/res/1268827.html
42
/mu/arch/2016-03-13/res/1268809.html
42
/mu/arch/2016-03-13/res/1268797.html
41
/mu/arch/2016-03-13/res/1268711.html
41
/mu/arch/2016-03-13/res/1268701.html
40
/mu/arch/2016-03-13/res/1268607.html
40
/mu/arch/2016-03-13/res/1268571.html
39
/mu/arch/2016-06-03/res/1268529.html
38
/mu/arch/2016-03-13/res/1268509.html
37
/mu/arch/2016-03-13/res/1268266.html
36
/mu/arch/2016-03-13/res/1268240.html
35
/mu/arch/2016-03-12/res/1268170.html
35
/mu/arch/2016-04-24/res/1268117.html
35
/mu/arch/2016-03-12/res/1268075.html
34
/mu/arch/2016-03-12/res/1268029.html
34
/mu/arch/2016-06-29/res/1267982.html
34
/mu/arch/2016-03-12/res/1267927.html
33
/mu/arch/2016-03-12/res/1267865.html
32
/mu/arch/2016-03-12/res/1267771.html
32
/mu/arch/2016-03-12/res/1267769.html
31


Разделим наши данны на тренировочые и тестовые. Пусть каждый десятый тред попадает в тест-сет.

In [5]:
data = []
test = []

it = 0
for topic in threads_by_topic:
    for thread in topic:
        full = []
        for post in thread:
            full.extend(post)
        it = it + 1
        if(it % 10 == 0):
            test.append(full)
        else:
            data.append(full)        

$\textbf{Задание.}$
В русском языке есть множество слов (частицы, междометия, всё что вы хотите), которые никак не отображают смысл слов и являются вспомогательными. Чтобы ваша модель работала лучше -- добавьте стоп-слова в список RUSSIAN_STOP_WORDS или в строку st_str. Эти слова отфильтруются из датасета перед тем, как модель начнет обучаться на датасете.

In [6]:
from gensim.utils import simple_preprocess
from gensim import corpora

RUSSIAN_STOP_WORDS = ['не', 'это', 'что','чем','это','как','https','нет','op','он','же','так','но','да','нет','или','и', 'на', "то", "бы", "все", "ты", "если", "по", "за", "там", "ну", "уже", "от", "есть","был", "даже", "было", "www", "com", "youtube", "из", "будет", "mp", "они", "только", "его", "она", "вот", 'просто', 'watch', 'кто', 'для', 'когда', 'тут', 'мне', 'где', 'мы', 'какой', 'может', 'меня', 'до', 'про', 'http', 'раз', 'почему', 'тебя', 'ещё', 'их', 'сейчас', 'тоже', 'во', 'чтобы', 'этого','без', 'него','вы','такой', 'можно', 'надо', 'нахуй', 'ли', 'потом', 'тред', 'больше', 'лучше', 'хуй', 'сам', 'после', 'со', 'лол', 'быть', 'нужно', 'этом', 'блять', 'бля', 'того', 'ничего', 'потому', 'нибудь', 'этот', 'под', 'через', 'ни', 'себе', 'ему', 'при', 'какие', 'пиздец', 'теперь', 'хоть', 'говно', 'тогда', 'блядь', 'кстати', 'че', 'себя', 'конечно', 'типа', 'много', 'том', 'нихуя', 'куда', 'всегда', 'нас', 'тот', 'ведь', 'эти', 'них', 'сука', 'пока', 'более', 'чего', 'html', 'были', 'всех', 'была', 'например', 'тем', 'ru', 'зачем', 'либо', 'вроде', 'всего', 'вопрос', 'php', 'против', 'здесь', 'ее', 'значит', 'совсем', 'сколько', 'им', 'org', 'именно', 'эту',]
st_str = "которых которые твой которой которого сих ком свой твоя этими слишком нами всему будь саму чаще ваше сами наш затем еще самих наши ту каждое мочь весь этим наша своих оба который зато те этих вся ваш такая теми ею которая нередко каждая также чему собой самими нем вами ими откуда такие тому та очень сама нему алло оно этому кому тобой таки твоё каждые твои мой нею самим ваши ваша кем мои однако сразу свое ними всё неё тех хотя всем тобою тебе одной другие этао само эта буду самой моё своей такое всею будут своего кого свои мог нам особенно её самому наше кроме вообще вон мною никто это"
RUSSIAN_STOP_WORDS.extend(st_str.split(' '))

data = [list(filter(lambda word: not word in RUSSIAN_STOP_WORDS, piece)) for piece in data]
id2word = corpora.Dictionary(data)

Создадим словарь и на его основе преобразуем слова в их id.

In [7]:
id2word = corpora.Dictionary(data)

# Create Corpus
texts = data

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]


Обучим LDA-модель, используя библиотеку gensim. Зададим число тем равно числу скачанных досок.

In [8]:
from gensim.models import LdaModel

model = LdaModel(corpus, id2word=id2word, num_topics=len(threads_by_topic))

Теперь получим топ-10 самых используемых в каждой теме слов.

$\textbf{Задание.}$
Оцените насколько хорошо модель разделила темы.

In [9]:
for i in range(len(threads_by_topic)):
    print([id2word[id[0]] for id in model.get_topic_terms(topicid = i, topn = 10)])

['лет', 'день', 'время', 'делать', 'the', 'бамп', 'назад', 'слушать', 'могу', 'два']
['день', 'делать', 'время', 'лет', 'могу', 'бамп', 'один', 'анон', 'хуйня', 'стоит']


Теперь прогоним тестовые треды на модели. Тестовый датасет разделен на n равных частей по 20 тредов, i-ая соответствует i-й доске.

In [10]:
other_corpus = [id2word.doc2bow(text) for text in [list(filter(lambda word: not word in RUSSIAN_STOP_WORDS, piece)) for piece in test]]

vector = [model[unseen_doc] for unseen_doc in other_corpus]
print(vector[0]) #вероятности принадлежности 0-го тестового треда в ту или иную тему

[(0, 0.2903671), (1, 0.7096329)]


In [11]:
i = 0

for res in vector:
    max_it = 0
    if(len(res) > 0):
        for it in range(1, len(res)):
            if(res[max_it][1] < res[it][1]):
                max_it = it
        print("Text #" + str(i) + ", topic #" + str(max_it) + str(", prob = " + str(res[max_it][1])))
    i = i + 1

Text #0, topic #1, prob = 0.7096329
Text #1, topic #1, prob = 0.5771407
Text #2, topic #0, prob = 0.590076
Text #3, topic #1, prob = 0.66211396
Text #4, topic #1, prob = 0.52184796
Text #5, topic #0, prob = 0.5786512
Text #6, topic #0, prob = 0.7713017
Text #7, topic #0, prob = 0.53100914
Text #8, topic #0, prob = 0.69227374
Text #9, topic #1, prob = 0.6162781
Text #10, topic #0, prob = 0.9732117
Text #11, topic #0, prob = 0.9650387
Text #12, topic #0, prob = 0.98139703
Text #13, topic #0, prob = 0.57826257
Text #14, topic #0, prob = 0.9478264
Text #15, topic #0, prob = 0.98636615
Text #16, topic #0, prob = 0.95515025
Text #17, topic #0, prob = 0.9853689
Text #18, topic #0, prob = 0.96847
Text #19, topic #0, prob = 0.96440005


$\textbf{Задание.}$

Оцените результаты работы модели на тест сете. Если модель разделили данные плохо -- объясните, почему?

## Часть 2. А теперь нормальный датасет.

А теперь давайте воспользуемся более стандартным датасетом библиотеки sklreatn -- 20newsgroups, посвященную статьям на различные темы. Выберем 7 -- Атеизм, яблочное железо, автомобили, хоккей, космос, христианство, ближний восток.

In [12]:
from sklearn.datasets import fetch_20newsgroups
categories = ['alt.atheism',
 'comp.sys.mac.hardware',
 'rec.autos',
 'rec.sport.hockey',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.mideast']
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'), categories = categories)

$\textbf{Задание}$

Найдите библиотечный или опишите свой список ENGSLISH_STOP_WORDS, убирающий не несущие никакого смысла английские слова.

In [13]:
from gensim.utils import simple_preprocess
from gensim import corpora
from nltk.corpus import stopwords

ENGLISH_STOP_WORDS = set(stopwords.words('english')) #TODO

print('the' in ENGLISH_STOP_WORDS) 

data = [list(filter(lambda word: not word in ENGLISH_STOP_WORDS, simple_preprocess(piece))) for piece in newsgroups_train.data]

True


$\textbf{Большое задание 1.}$

Для списка data создайте словарь id2word. Получите преобразованный TermDocumentFrequency список corpust и обучите на нем LDA модель.

In [14]:
from gensim.models import LdaModel, LsiModel

print(data[0])
id2word = corpora.Dictionary(data)

# Create Corpus
texts = data

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

model = LdaModel(corpus, id2word = id2word, num_topics = len(categories))

['considering', 'adding', 'floptical', 'drive', 'current', 'system', 'would', 'like', 'know', 'floptical', 'drives', 'recommended', 'quality', 'performance', 'preference', 'would', 'floptical', 'drives', 'capable', 'handling', 'floppies', 'handling', 'floppies', 'necessity', 'far', 'know', 'bit', 'iomega', 'floptical', 'infinity', 'floptical', 'drives', 'comments', 'recommendations', 'either', 'floptical', 'drives', 'worth', 'looking', 'purchased', 'mail', 'order', 'places', 'etc', 'thanks', 'advance', 'please', 'send', 'replies', 'directly', 'umsoroko', 'ccu', 'umanitoba', 'ca']
[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 4), (12, 1), (13, 1), (14, 1), (15, 2), (16, 6), (17, 2), (18, 1), (19, 1), (20, 2), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 2)]]


In [15]:
#Выведем получившийся список тем:
for i in range(len(categories)):
    print([id2word[id[0]] for id in model.get_topic_terms(topicid = i, topn = 10)])

['would', 'team', 'lindros', 'go', 'surprise', 'patrick', 'rangers', 'trade', 'ottawa', 'washington']
['finnish', 'blues', 'conform', 'checking', 'one', 'game', 'hawks', 'playoffs', 'fingers', 'first']
['turkey', 'caucasus', 'kurds', 'armenian', 'volunteers', 'ability', 'russian', 'source', 'moslems', 'power']
['would', 'people', 'think', 'go', 'one', 'nothing', 'smileys', 'reasonable', 'beezer', 'sarcasm']
['people', 'one', 'think', 'would', 'failure', 'gt', 'chart', 'many', 'never', 'said']
['rate', 'abstinence', 'sex', 'jen', 'aids', 'use', 'bottom', 'slot', 'failure', 'top']
['bible', 'people', 'would', 'tony', 'christian', 'many', 'one', 'laws', 'someone', 'point']


$\textbf{Большое задание 2.}$

В соответствии с тренировочными, обработайте тестовые данные.

Напишите функцию, которая с помощью модели возвращает наиболее вероятный id темы. С помощью F-меры оцените правильность работы модели.

In [17]:
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'), categories = categories)

#TODO: YOUD CODE
other_corpus = [id2word.doc2bow(text) for text in [list(filter(lambda word: not word in ENGLISH_STOP_WORDS, simple_preprocess(piece))) for piece in newsgroups_test.data]]
vector = [model[unseen_doc] for unseen_doc in other_corpus]


In [18]:
def textid(text):
    res = vector[text]
    max_it = 0
    if(len(res) > 0):
        for it in range(1, len(res)):
            if(res[max_it][1] < res[it][1]):
                max_it = it
        print("Text #" + str(text) + ", topic #" + str(max_it))

In [24]:
textid(123)

Text #123, topic #1


In [25]:
#(correct) target values
newsgroups_test.target

array([5, 0, 6, ..., 6, 1, 6], dtype=int64)

In [26]:
#Estimated targets
i = 0
topics = []
for res in vector:
    max_it = 0
    if(len(res) > 0):
        for it in range(1, len(res)):
            if(res[max_it][1] < res[it][1]):
                max_it = it
        topics.append(max_it)
    i = i + 1

In [27]:
from sklearn.metrics import f1_score

f1_score(newsgroups_test.target, topics, average = 'micro')

0.13385826771653545

In [35]:
import itertools
permuts = list(itertools.permutations([0, 1, 2, 3, 4, 5, 6]))
real_topics = newsgroups_test["target"].tolist()

In [29]:
slyce = [0, 6, 1, 12, 16, 203, 641]
real_slice = []
for i in slyce:
    real_slice.append(real_topics[i])

res = {}                                                                   
for per in permuts:
    res[per] = f1_score(real_slice, per, average = "micro")

In [30]:
print(res)

{(0, 1, 2, 3, 4, 5, 6): 0.14285714285714285, (0, 1, 2, 3, 4, 6, 5): 0.2857142857142857, (0, 1, 2, 3, 5, 4, 6): 0.14285714285714285, (0, 1, 2, 3, 5, 6, 4): 0.14285714285714285, (0, 1, 2, 3, 6, 4, 5): 0.42857142857142855, (0, 1, 2, 3, 6, 5, 4): 0.2857142857142857, (0, 1, 2, 4, 3, 5, 6): 0.2857142857142857, (0, 1, 2, 4, 3, 6, 5): 0.42857142857142855, (0, 1, 2, 4, 5, 3, 6): 0.2857142857142857, (0, 1, 2, 4, 5, 6, 3): 0.2857142857142857, (0, 1, 2, 4, 6, 3, 5): 0.5714285714285714, (0, 1, 2, 4, 6, 5, 3): 0.42857142857142855, (0, 1, 2, 5, 3, 4, 6): 0.14285714285714285, (0, 1, 2, 5, 3, 6, 4): 0.14285714285714285, (0, 1, 2, 5, 4, 3, 6): 0.14285714285714285, (0, 1, 2, 5, 4, 6, 3): 0.14285714285714285, (0, 1, 2, 5, 6, 3, 4): 0.2857142857142857, (0, 1, 2, 5, 6, 4, 3): 0.2857142857142857, (0, 1, 2, 6, 3, 4, 5): 0.2857142857142857, (0, 1, 2, 6, 3, 5, 4): 0.14285714285714285, (0, 1, 2, 6, 4, 3, 5): 0.2857142857142857, (0, 1, 2, 6, 4, 5, 3): 0.14285714285714285, (0, 1, 2, 6, 5, 3, 4): 0.1428571428571428

In [34]:
import operator
res_perm = max(res.items(), key = operator.itemgetter(1)) [0]
res[res_perm]

0.7142857142857143

In [32]:
for i in range(len(topics)):
    if topics[i] == 0:
        topics[i] = 3
    elif topics[i] == 3:
        topics[i] = 4
    elif topics[i] == 4:
        topics[i] = 6
    elif topics[i] == 5:
        topics[i] = 0
    elif topics[i] == 6:
        topics[i] = 5

In [ ]:
from sklearn.metrics import f1_score
f1_score(real_topics, topics, average = "micro")